In [53]:
import pandas as pd

In [54]:
import csv
import json

In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
path = '/content/drive/MyDrive/Study/Elice/project2'

In [57]:
import matplotlib.pyplot as plt

# 한글 폰트 설정
!apt -qq -y install fonts-nanum                 # 상기 라이브러리에 폰트 설치 
                                                # 이하 기본 폰트가 되도록 설정
# 한글 폰트를 사용할 수 있게 만들어줌
import matplotlib as mpl
import matplotlib.font_manager as fm
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') # 그래프에서 한글 깨지는 현상 방지
plt.rc('axes', unicode_minus=False) # 그래프에서 마이너스 기호가 표시되도록
mpl.font_manager._rebuild()

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [68]:
with open(f'{path}/TMDB_base_info_1_1000.json', 'r', encoding='utf-8') as input_file, open(f'{path}/TMDB_base_info_1_1000.csv', 'w', newline = '') as output_file:
  data = json.load(input_file)
  f = csv.writer(output_file)
  f.writerow(['title', 'original_title', 'overview'])

  for datum in data:
    title = data[datum]['title']
    original_title = data[datum]['original_title']
    overview = data[datum]['overview']
    f.writerow([title, original_title, overview])
    
# title = data['2']['title']
# original_title = data['2']['original_title']
# overview = data['2']['overview']
# #print(json.dumps(data, indent='\t'))
# print(title)
# print(original_title)
# print(overview)
    

In [69]:
data1 = pd.read_csv(f'{path}/TMDB_base_info_1_1000.csv')
print(data1.head())

                              title  ...                                           overview
0                               아리엘  ...  주인공 카스리넨은 광부다. 일하던 탄광이 폐광이 되며 도산을 하고 사장은 유일하게 ...
1                           천국의 그림자  ...  첫 장면부터 청소차가 새벽을 가른다. 청소부는 제각기 정해진 구역에서 익숙하게 쓰레...
2                                포룸  ...  새해 전날 밤, LA 몽시뇰 호텔에서 첫근무를 하게 된 괴짜 벨보이 테드는 상상조차...
3                            킬러 나이트  ...  어느날 프랭크(Frank Wyatt: 에밀리오 에스테베즈 분)와 마이크(Mike P...
4  Life in Loops (A Megacities RMX)  ...                                                NaN

[5 rows x 3 columns]


In [70]:
from math import log # IDF(역 문서 빈도) 계산용

In [71]:
# 코사인 유사도 계산
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

In [72]:
# TF-IDF 및 코사인 유사도, 줄거리 기반 영화 추천용
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [73]:
# 결측치 확인
data1['overview'].isnull().sum()

117

In [74]:
# 결측치 채우기 ''
data1['overview'] = data1['overview'].fillna('')

In [75]:
# 결측치 재확인
data1['overview'].isnull().sum()

0

In [76]:
tfidf = TfidfVectorizer(stop_words='english')
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data1['overview'])
print(tfidf_matrix.shape)
data1.head()

(822, 29160)


,title,original_title,overview
0,아리엘,Ariel,주인공 카스리넨은 광부다. 일하던 탄광이 폐광이 되며 도산을 하고 사장은 유일하게 ...
1,천국의 그림자,Varjoja paratiisissa,첫 장면부터 청소차가 새벽을 가른다. 청소부는 제각기 정해진 구역에서 익숙하게 쓰레...
2,포룸,Four Rooms,"새해 전날 밤, LA 몽시뇰 호텔에서 첫근무를 하게 된 괴짜 벨보이 테드는 상상조차..."
3,킬러 나이트,Judgment Night,어느날 프랭크(Frank Wyatt: 에밀리오 에스테베즈 분)와 마이크(Mike P...
4,Life in Loops (A Megacities RMX),Life in Loops (A Megacities RMX),


In [79]:
# 문서의 코사인 유사도 구하기
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

indices = pd.Series(data1.index, index=data1['title']).drop_duplicates()
print(indices.head())

title
아리엘                                 0
천국의 그림자                             1
포룸                                  2
킬러 나이트                              3
Life in Loops (A Megacities RMX)    4
dtype: int64


In [82]:
def get_recommendations(title, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[title]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    movie_indices = [i[0] for i in sim_scores]

    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data1['title'].iloc[movie_indices]

In [84]:
get_recommendations('비포 선라이즈')

684    열차 안의 낯선 자들
282          지상의 밤
592            얼굴들
104       브로크백 마운틴
371    우리에게 내일은 없다
401              불
46           비포 선셋
388        저수지의 개들
3           킬러 나이트
71            프레데터
Name: title, dtype: object

In [90]:
data1.iloc[0]['overview']

'주인공 카스리넨은 광부다. 일하던 탄광이 폐광이 되며 도산을 하고 사장은 유일하게 남은 것은 권총과 승용차 한대. 차 키를 카스리넨에게 주고 자신은 권콩으로 자살을 한다. 실업자가 되어 달랑 차 한 대만 끌고 도시로 온 그는 주차 단속을 하던 필라야와 너무 쉽게 잠을 자고 곧 동거를 시작하는데 그들의 필요 충분 조건은 사랑뿐 이었을 뿐이다. 그녀에겐 딸린 아들도 있다. 가진 것 없고 사랑만 있는 이들 앞엔 역경만 닥쳐오고 결국 갖은 우여곡절 끝에 그들은 현실을 도피해 밀항선을 타고 꿈의 땅 아리엘로 떠난다.'